# Regression

Regression is a function approximation problem in which we seek to learn a mapping such that $f(\mathbf{y},\boldsymbol{\theta}) \approx \mathbf{c}$ for all input-target pairs $(\mathbf{y},\mathbf{c}) \in \mathcal{D}$.  Here, $\mathbf{y}$ represents the inputs (e.g., parameters) and $\mathbf{c}$ represents values of an unknown mapping (e.g., measurements, observations, etc.)

### Outcomes 
In this tutorial, you will

*   generate regression test problems to approximate one- or two-dimensional functions
*   learn how to construct a neural network using PyTorch
*   train a network to fit your function using stochastic optimization

### Suggested Activities

*  How does the network architecture affect the performance?  Play with the width, depth, and activation functions and see if there are differences in approximation quality and training challenges.  
*  How does the data affect performance?  What if we have fewer training data points?  Do we fit better?  How does the noise level affect our performance?
*  How does the optimizer affect performance?  Try different optimizer parameters (e.g., learning rate, momentum, etc.).  Compare different optimizers and see which ones train best.

* **Challenges:** 

  * How does the neural network approximation compare to traditional data fitting approaches? Is polynomial approximation better?
  * How well do we generalize (in this case, extrapolate) outside of the domain?
  * What if the function we seek to approximate is outside of the hypothesis space of functions (e.g., if we approximate a rational function)?  Can a neural network capture this behavior?  
  * We often try to add regularization to avoid fitting noise.  Try to implement Tikhonov regularization from scratch.  This is encoded in the optimizers with the ```weight_decay``` parameter, which gives you some to which to compare your implmentation.

Check out the [PyTorch Documentation](https://pytorch.org/docs/stable/index.html) as you explore!








## Step 1: Import Packages

We start by importing the necessary packages to run our code.  We are installing the following packages:

   * deep learning toolbox [Pytorch](https://pytorch.org/)
   * visualization toolbox [Matplotlib](https://matplotlib.org/)
   * DNN101 repository [https://github.com/elizabethnewman/dnn101](https://github.com/elizabethnewman/dnn101).

In [ ]:
!python -m pip install git+https://github.com/elizabethnewman/dnn101.git

In [2]:
import dnn101
import torch
import torch.nn as nn
import matplotlib as mpl
import matplotlib.pyplot as plt

## Step 2: Create the Data

We provide two approaches to creating data:
1. **DNN101DataRegression1D**: fit a one-dimensional function $f: [a_1, b_1] \to \mathbb{R}$
2. **DNN101DataRegression2D**: fit a two-dimensional function $f: [a_1, b_1] \times [a_2, b_2] \to \mathbb{R}$

The user provides a callable function $f$; e.g., 
```python
f = lambda x: x ** 2                      # one-dimensional
f = lambda x: x[:,0] ** 2 + x[:, 1] ** 2  # two-dimensional
```

The **DNN101Data** environments provide methods to plot the data and the network predictions.


In [ ]:
from dnn101.regression import DNN101DataRegression1D, DNN101DataRegression2D

# set seed for reproducibility
torch.manual_seed(123)

# data parameters
n_train = 2000      # number of training points
n_val   = 100       # number of validation points
n_test  = 100       # number of test points
sigma   = 0.2       # noise level


# function to approximate
f = lambda x: torch.sin(x)
domain  = (-3, 3)   # domain of function
dataset = DNN101DataRegression1D(f, domain, noise_level=sigma)

# f = lambda x: torch.sin(x[:, 0]) + torch.cos(x[:, 1])
# domain  = [-3, 3, -3, 3]   # domain of function
# dataset = DNN101DataRegression2D(f, domain, noise_level=sigma)

# generate data
x, y = dataset.generate_data(n_train + n_val + n_test)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset.split_data(x, y, n_train=n_train, n_val=n_val)

# plot data
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['lines.linewidth'] = 8
mpl.rcParams['font.size'] = 10

dataset.plot_data((x_train, y_train), (x_val, y_val), (x_test, y_test))
plt.show()
mpl.rcParams.update(mpl.rcParamsDefault)

## Step 3: Train!

This block is the heart of DNN training and consists of four main steps
1. Define the architecture of the DNN (for this notebook, we choose [Linear Layers](https://pytorch.org/docs/stable/nn.html#linear-layers) and [Activation Functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity))
2. Choose a loss function ([PyTorch Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)).  For regression, we use the [Mean Squared Error Loss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss). 
3. Choose an optimizer ([PyTorch Optimizers](https://pytorch.org/docs/stable/optim.html?highlight=optim#torch.optim.Optimizer))
4. Train with stochastic optimization.  

If you change parameters, be sure to run all blocks in this section to make sure everything is connected properly.

### 1. Architecture

We will construct a fully-connected neural network using [linear layers](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) and [activation functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity). 

Note that in PyTorch, a linear layer is, by default, an affine transformation.

In [4]:
net = nn.Sequential(
    nn.Linear(x_train.shape[1], 14),
    nn.Tanh(),
    nn.Linear(14, 10),
    nn.Sigmoid(),
    nn.Linear(10, y_train.shape[1])
)

### 2. Loss Function

For function approximation, our goal is to minimize the expected least squares loss given by
\begin{align*}
\min_{\boldsymbol{\theta}} \mathbb{E}\|F(\mathbf{y}, \boldsymbol{\theta}) - \mathbf{c}\|_2^2
\end{align*}

Note: in PyTorch, the default [mean squared error loss function](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html)  is given by
\begin{align*}
\frac{1}{n_b}\sum_{i=1}^{n_b} \tfrac{1}{m}\|\mathbf{z}_i - \mathbf{c}_i\|_2^2
\end{align*}
where $n_b$ is the batch size and $m$ is the number of target features.  Effectively, the loss averages over all of the entries rahter than all of the samples.

In [5]:
loss = torch.nn.MSELoss()

### 3. Optimizer

There are many choices of [PyTorch optimizers](https://pytorch.org/docs/stable/optim.html).  We select the popular Adam (Adapative Momentum estimation) method.

In [6]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

### 4. Train!

In [ ]:
from IPython import display
mpl.rcParams['figure.figsize'] = (16, 6)
mpl.rcParams['lines.linewidth'] = 8
mpl.rcParams['font.size'] = 10

# import training
from dnn101.utils import evaluate

# printing and plotting options (only one can be True)
verbose = True             # printouts
show_plot = not verbose     # plot to show training

# set seed for reproducibility
torch.manual_seed(42)

# store and print results
info = {
    'headers': ('epoch', 'running_loss', 'training_loss', 'validation_loss'),
    'formats': '{:<15d}{:<15.4e}{:<15.4e}{:<15.4e}',
    'values': None
    }

loss_train, _ = evaluate(net, loss, (x_train, y_train))
loss_val, _ = evaluate(net, loss, (x_val, y_val))
info['values'] = [[-1, 0.0, loss_train, loss_val]]

if verbose:
    print(('{:<15s}' * len(info['headers'])).format(*info['headers']))
    print(info['formats'].format(*info['values'][-1]))

# ============================================================================ #
# OUTER ITERATION
# ============================================================================ #
max_epochs = 20
batch_size = 5
for epoch in range(max_epochs):

    # ======================================================================== #
    # INNER ITERATION (update for all batches in one epoch)
    # ======================================================================== #
    n_batch = x_train.shape[0] // batch_size
    shuffle_idx = torch.randperm(x_train.shape[0])
    running_loss = 0.0
    for i in range(n_batch):
        # select batch
        idx = shuffle_idx[i * batch_size:(i + 1) * batch_size]
        xb, yb = x_train[idx], y_train[idx]

        # zero out gradients (very important step!)
        optimizer.zero_grad()

        # forward propagate
        yb_hat = net(xb)

        # evaluate (with average loss)
        phi = loss(yb_hat, yb)
        running_loss += yb.numel() * phi.item()

        # backward propagate (with automatic differentiation)
        phi.backward()

        # update (with optimizer rule)
        optimizer.step()

    # evaluate performance for each epoch
    loss_running = running_loss / (n_batch * batch_size)
    loss_train, _ = evaluate(net, loss, (x_train, y_train))
    loss_val, _ = evaluate(net, loss, (x_val, y_val))

    # store and print or plot results
    info['values'].append([epoch, loss_running, loss_train, loss_val])
    if verbose:
        print(info['formats'].format(*info['values'][-1]))

    # plot function approximation and loss
    if show_plot:
      with torch.no_grad():
        plt.subplot(1, 2, 1)
        dataset.plot_prediction(dataset.f)
        dataset.plot_prediction(net, label='pred', color='g', linestyle='--')
        plt.title('approx: epoch = %d' % epoch)

        plt.subplot(1, 2, 2)
        values = torch.tensor(info['values'])

        plt.semilogy(values[:, 0], values[:, 2], label='train')
        plt.semilogy(values[:, 0], values[:, 3], '--', label='validation')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.xlim([-1, max_epochs])
        plt.ylim([1e-2, 1e0])
        plt.legend()
        plt.title('convergence: test loss = %0.4e' % evaluate(net, loss, (x_test, y_test))[0])


        display.display(plt.gcf())
        display.clear_output(wait=True)
        plt.clf()


mpl.rcParams.update(mpl.rcParamsDefault)

## Step 4: Inference

Let's assess how well our neural network performs on unseen data (i.e., the test data) and how well it extrpolates outside of the domain on which we trained.

In [ ]:
# final losses
loss_train, _ = evaluate(net, loss, (x_train, y_train))
loss_val, _ = evaluate(net, loss, (x_val, y_val))
loss_test, _ = evaluate(net, loss, (x_test, y_test))

print('Train Loss = %0.4e' % loss_train)
print('Valid Loss = %0.4e' % loss_val)
print('Test Loss = %0.4e' % loss_test)

In [ ]:
# extrapolation

mpl.rcParams['lines.linewidth'] = 8
mpl.rcParams['font.size'] = 10

# create larger domain (extend by n_ext in every direction)
n_ext = 4

if isinstance(dataset, DNN101DataRegression1D):
    mpl.rcParams['figure.figsize'] = (8, 6)
    domain_ext = (domain[0] - n_ext, domain[1] + n_ext)
    dataset_ext = DNN101DataRegression1D(f, domain_ext, noise_level=sigma)
    
    plt.vlines(domain[0], -1, 1, 'k', ':', label='domain')
    plt.vlines(domain[1], -1, 1, 'k', ':')

    # plot prediction
    dataset_ext.plot_prediction(dataset.f)
    dataset_ext.plot_prediction(net, label='pred', color='g', linestyle='--')
    plt.legend()
    plt.show()

else:
    mpl.rcParams['figure.figsize'] = (16, 6)
    domain_ext = (domain[0] - n_ext, domain[1] + n_ext, 
                  domain[2] - n_ext, domain[3] + n_ext)
    dataset_ext = DNN101DataRegression2D(f, domain_ext, noise_level=sigma)

    plt.subplot(1, 3, 1)
    dataset_ext.plot_prediction(dataset.f)
    plt.vlines(domain[0], domain[2], domain[3], 'k')
    plt.vlines(domain[1], domain[2], domain[3], 'k')
    plt.hlines(domain[2], domain[0], domain[1], 'k')
    plt.hlines(domain[3], domain[0], domain[1], 'k')
    plt.title('true')

    plt.subplot(1, 3, 2)
    dataset_ext.plot_prediction(net)
    plt.vlines(domain[0], domain[2], domain[3], 'k')
    plt.vlines(domain[1], domain[2], domain[3], 'k')
    plt.hlines(domain[2], domain[0], domain[1], 'k')
    plt.hlines(domain[3], domain[0], domain[1], 'k')
    plt.title('prediction')

    plt.subplot(1, 3, 3)
    dataset_ext.plot_prediction1(lambda x: torch.abs(net(x) - dataset.f(x)))
    plt.vlines(domain[0], domain[2], domain[3], 'k')
    plt.vlines(domain[1], domain[2], domain[3], 'k')
    plt.hlines(domain[2], domain[0], domain[1], 'k')
    plt.hlines(domain[3], domain[0], domain[1], 'k')
    plt.title('abs. diff.')


    plt.show()

mpl.rcParams.update(mpl.rcParamsDefault)